In [12]:
import numpy as np
import parselmouth
from parselmouth.praat import call

In [31]:
def feature_extract(voiceID, f0min, f0max, unit):
    sound = parselmouth.Sound(voiceID)
    duration = call(sound, "Get total duration") # get duration
    pitch = call(sound, "To Pitch", 0.0, f0min, f0max) # create praat pitch object
    meanF0 = call(pitch, "Get mean", 0, 0, unit) # get mean pitch
    stdevF0 = call(pitch, "Get standard deviation", 0, 0, unit) # get standard deviation
    pointProcess = call(sound, "To PointProcess (periodic, cc)", f0min, f0max)
    localJitter = call(pointProcess, "Get jitter (local)", 0, 0, 0.0001, 0.02, 1.3) # local jitter (%)
    localabsoluteJitter = call(pointProcess, "Get jitter (local, absolute)", 0, 0, 0.0001, 0.02, 1.3) # local absolute jitter (s)
    rapJitter = call(pointProcess, "Get jitter (rap)", 0, 0, 0.0001, 0.02, 1.3) # rap jitter
    ppq5Jitter = call(pointProcess, "Get jitter (ppq5)", 0, 0, 0.0001, 0.02, 1.3) # ppq5 jitter 
    localShimmer = call([sound, pointProcess], 'Get shimmer (local)', 0, 0, 0.0001, 0.02, 1.3, 1.6)
    localdbShimmer = call([sound, pointProcess], 'Get shimmer (local_dB)', 0, 0, 0.0001, 0.02, 1.3, 1.6)
    apq11Shimmer = call([sound, pointProcess], "Get shimmer (apq11)", 0, 0, 0.0001, 0.02, 1.3, 1.6)
    harmonicity = call(sound, "To Harmonicity (cc)", 0.01, f0min, 0.1, 4.5)
    hnr = call(harmonicity, "Get mean", 0, 0)
    
    # to percent
    localJitter *= 100. 
    rapJitter *= 100
    ppq5Jitter *= 100
    localShimmer *= 100
    apq11Shimmer *= 100
    
    return duration, meanF0, stdevF0, localJitter, localabsoluteJitter, rapJitter, ppq5Jitter, localShimmer, localdbShimmer, apq11Shimmer, hnr

In [34]:
src = 'train_data/mild/1/chapter1.wav'
src = 'C:/Users/jae/Music/hearing_loss_split/mild/7/chapter1.wav'
sound = parselmouth.Sound(src)
A = feature_extract(sound, 75, 1000, "Hertz")

In [35]:
normal_feat = []
mild_feat = []
severe_feat = []



(15.564444444444444, 248.15835100394648, 48.964759586498786, 0.23595393061140862, 9.524669341792474e-06, 0.11666211308024407, 0.12221838642286278, 0.030910566509951904, 28.261161457376012, 3.6927855212913037, 20.422202343985752)
